# 0. Packages and Functions

## 0.1. Packages

In [ ]:
import pandas as pd
import string
from nltk import ngrams
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
import pyarrow.parquet as pq
import pyarrow as pyarrow
import numpy as np

## 0.2. Functions

In [ ]:
#pre-process text for lexicon based approaches
def preprocess_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # convert to lower case
    text = text.lower()
    # remove blank spaces
    text = ' '.join(text.split())
    # remove newline characters
    text = text.replace('\n', '')
    return text

### Absolute Count with Frequency

In [195]:
#Create function that take the dataframe, lexicon and n-gram value (how many n-grams should be considered) and determine the 
#count of words in dataframe text that match the lexicon
def count_lexicon_words_1(text_df, lexicon):
    lexicon = lexicon["Lexicon"]
    count = []

    for text in text_df["Text"]:
        lexicon_counts = 0

        for word in lexicon:
            lexicon_counts += text.lower().count(word.lower())
            
        count.append(lexicon_counts)
        
    text_df["Lexicon Count"] = count
    
    return(text_df)

#create a function that used the lexicon approach to determine with the target is yes or no
def lexicon_target_classifier_1(df, treshold):
    target = []
    
    count = df["Lexicon Count"]
    
    for c in count:
        if c < treshold:
            target.append("No")
        else:
            target.append("Yes")
            
    df["Target Lexicon"] = target
    return(df)

#Combine both functions to classify articles based on the lexicon
def lexicon_climate_classifier_1(text_df, lexicon, treshold):
    df = count_lexicon_words_1(text_df, lexicon)
    
    return(lexicon_target_classifier_1(df, treshold))

def threshold_metrics_1(df_text, lexicon, min_treshhold, max_treshhold):
    for i in range(min_treshhold, max_treshhold + 1):
        df = lexicon_climate_classifier_1(df_text, lexicon, i)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)
        
        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

        # print the metrics
        print("Threshhold:", i)
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 score:", f1_score)
        print(cross_table)
        print("\n")
        
def get_metrics_df_1(df_text, lexicon, min_treshhold, max_treshhold, Lexicon_name):    
    Accuracy = []
    Precision = []
    Recall = []
    F1_score = []
    Name = []
    Treshhold = []
    Technique = []
    
    for i in range(min_treshhold, max_treshhold + 1):
        df = lexicon_climate_classifier_1(df_text, lexicon, i)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        
        Accuracy.append(accuracy)
        Precision.append(precision)
        Recall.append(recall)
        F1_score.append(f1_score)
        Name.append(Lexicon_name)
        Treshhold.append(i)
        Technique.append("Absolute Frequency")
        
    return(pd.DataFrame({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score}))
    


In [ ]:
#Create function that take the dataframe, lexicon and n-gram value (how many n-grams should be considered) and determine the 
#count of words in dataframe text that match the lexicon
def count_lexicon_words_1(text_df, lexicon):
    lexicon = lexicon["Lexicon"]
    count = []

    for text in text_df["Text"]:
        lexicon_counts = 0

        for word in lexicon:
            lexicon_counts += text.lower().count(word.lower())
        
        count.append(lexicon_counts)
        
    text_df["Lexicon Count"] = count
    
    return(text_df)

In [ ]:
count_lexicon_words_1(tag_climate_df, BBC_Lexicon)

### Relative Count with Frequency

In [ ]:
#Create function that take the dataframe, lexicon and n-gram value (how many n-grams should be considered) and determine the 
#count of words in dataframe text that match the lexicon
def count_lexicon_words_2(text_df, lexicon):
    lexicon = lexicon["Lexicon"]
    count = []
    
    for text in text_df["Text"]:
        lexicon_counts = 0

        for word in lexicon:
            lexicon_counts += text.lower().count(word.lower())
        
        word_list = text.split() 
        word_count = len(word_list)
        count.append((lexicon_counts/word_count)*100)

    text_df["Lexicon Count"] = count
    
    return(text_df)

#create a function that used the lexicon approach to determine with the target is yes or no
def lexicon_target_classifier_2(df, treshold):
    target = []
    
    count = df["Lexicon Count"]
    
    for c in count:
        if c < treshold:
            target.append("No")
        else:
            target.append("Yes")
            
    df["Target Lexicon"] = target
    return(df)

#Combine both functions to classify articles based on the lexicon
def lexicon_climate_classifier_2(text_df, lexicon, treshold):
    df = count_lexicon_words_2(text_df, lexicon)
    
    return(lexicon_target_classifier_2(df, treshold))

def threshold_metrics_2(df_text, lexicon, min_treshhold, max_treshhold, jump):
    
    if(jump == 0):
        df = lexicon_climate_classifier_2(df_text, lexicon, min_threshhold)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

        # print the metrics
        print("Threshhold:", i)
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 score:", f1_score)
        print(cross_table)
        print("\n")

    else:   
        for num in range(int((max_treshhold - min_treshhold) / jump) + 1):
            i = min_treshhold + num * jump
            df = lexicon_climate_classifier_2(df_text, lexicon, i)
            cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

            # calculate classification metrics using scikit-learn
            accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
            precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
            recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

            # print the metrics
            print("Threshhold:", i)
            print("Accuracy:", accuracy)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 score:", f1_score)
            print(cross_table)
            print("\n")

def get_metrics_df_2(df_text, lexicon, min_treshhold, max_treshhold, jump, Lexicon_name):    
    Accuracy = []
    Precision = []
    Recall = []
    F1_score = []
    Name = []
    Treshhold = []
    Technique = []
    
    for num in range(int((max_treshhold - min_treshhold) / jump) + 1):
        i = min_treshhold + num * jump
        df = lexicon_climate_classifier_2(df_text, lexicon, i)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        
        Accuracy.append(accuracy)
        Precision.append(precision)
        Recall.append(recall)
        F1_score.append(f1_score)
        Name.append(Lexicon_name)
        Treshhold.append(i)
        Technique.append("Relative Frequency")
        
    return(pd.DataFrame({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score}))


### Absolute Term Presences

In [ ]:
#Create function that take the dataframe, lexicon and n-gram value (how many n-grams should be considered) and determine the 
#count of words in dataframe text that match the lexicon
def count_lexicon_words_3(text_df, lexicon):
    lexicon = lexicon["Lexicon"]
    count = []

    for text in text_df["Text"]:
        lexicon_counts = 0

        for word in lexicon:
            if text.lower().count(word.lower()) > 0:
                lexicon_counts += 1
        
        count.append(lexicon_counts)
        
    text_df["Lexicon Count"] = count
    
    return text_df

#create a function that used the lexicon approach to determine with the target is yes or no
def lexicon_target_classifier_3(df, treshold):
    target = []
    
    count = df["Lexicon Count"]
    
    for c in count:
        if c < treshold:
            target.append("No")
        else:
            target.append("Yes")
            
    df["Target Lexicon"] = target
    return(df)

#Combine both functions to classify articles based on the lexicon
def lexicon_climate_classifier_3(text_df, lexicon, treshold):
    df = count_lexicon_words_3(text_df, lexicon)
    
    return(lexicon_target_classifier_3(df, treshold))

def threshold_metrics_3(df_text, lexicon, min_treshhold, max_treshhold):
    for i in range(min_treshhold, max_treshhold + 1):
        df = lexicon_climate_classifier_3(df_text, lexicon, i)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)
        
        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

        # print the metrics
        print("Threshhold:", i)
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 score:", f1_score)
        print(cross_table)
        print("\n")
        
def get_metrics_df_3(df_text, lexicon, min_treshhold, max_treshhold, Lexicon_name):    
    Accuracy = []
    Precision = []
    Recall = []
    F1_score = []
    Name = []
    Treshhold = []
    Technique = []
    
    for i in range(min_treshhold, max_treshhold + 1):
        df = lexicon_climate_classifier_3(df_text, lexicon, i)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        
        Accuracy.append(accuracy)
        Precision.append(precision)
        Recall.append(recall)
        F1_score.append(f1_score)
        Name.append(Lexicon_name)
        Treshhold.append(i)
        Technique.append("Absolute Presences")
        
    return(pd.DataFrame({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score}))
    


### Relative Term Presences

In [ ]:
#Create function that take the dataframe, lexicon and n-gram value (how many n-grams should be considered) and determine the 
#count of words in dataframe text that match the lexicon
def count_lexicon_words_4(text_df, lexicon):
    lexicon = lexicon["Lexicon"]
    count = []

    for text in text_df["Text"]:
        lexicon_counts = 0

        for word in lexicon:
            if text.lower().count(word.lower()) > 0:
                lexicon_counts += 1
        
        count.append(lexicon_counts)
        
    text_df["Lexicon Count"] = count
    
    return text_df

#create a function that used the lexicon approach to determine with the target is yes or no
def lexicon_target_classifier_4(df, treshold):
    target = []
    
    count = df["Lexicon Count"]
    
    for c in count:
        if c < treshold:
            target.append("No")
        else:
            target.append("Yes")
            
    df["Target Lexicon"] = target
    return(df)

#Combine both functions to classify articles based on the lexicon
def lexicon_climate_classifier_4(text_df, lexicon, treshold):
    df = count_lexicon_words_4(text_df, lexicon)
    
    return(lexicon_target_classifier_4(df, treshold))

def threshold_metrics_4(df_text, lexicon, min_treshhold, max_treshhold, jump):
    
    if(jump == 0):
        df = lexicon_climate_classifier_4(df_text, lexicon, min_treshhold)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

        # print the metrics
        print("Threshhold:", i)
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 score:", f1_score)
        print(cross_table)
        print("\n")
        
    else:
        for num in range(int((max_treshhold - min_treshhold) / jump) + 1):
            i = min_treshhold + num * jump
            df = lexicon_climate_classifier_4(df_text, lexicon, i)
            cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

            # calculate classification metrics using scikit-learn
            accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
            precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
            recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

            # print the metrics
            print("Threshhold:", i)
            print("Accuracy:", accuracy)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 score:", f1_score)
            print(cross_table)
            print("\n")

def get_metrics_df_4(df_text, lexicon, min_treshhold, max_treshhold, jump, Lexicon_name):    
    Accuracy = []
    Precision = []
    Recall = []
    F1_score = []
    Name = []
    Treshhold = []
    Technique = []
    
    for num in range(int((max_treshhold - min_treshhold) / jump) + 1):
        i = min_treshhold + num * jump
        df = lexicon_climate_classifier_4(df_text, lexicon, i)
        cross_table = pd.crosstab(df['Target'], df['Target Lexicon'], margins=True)

        # calculate classification metrics using scikit-learn
        accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
        precision = cross_table.iloc[1,1] / cross_table.iloc[0,1] if cross_table.iloc[0,1] != 0 else 0
        recall = cross_table.iloc[1,1] / cross_table.iloc[1,0] if cross_table.iloc[1,0] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        
        Accuracy.append(accuracy)
        Precision.append(precision)
        Recall.append(recall)
        F1_score.append(f1_score)
        Name.append(Lexicon_name)
        Treshhold.append(i)
        Technique.append("Relative Presences")
        
    return(pd.DataFrame({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score}))


Accuracy: This metric measures the overall performance of a model. It is defined as the number of correct predictions divided by the total number of predictions. Accuracy is a good metric to use when the classes are roughly balanced, meaning there are about the same number of positive and negative examples in the dataset.

Precision: This metric measures how many of the positive predictions made by a model are actually correct. It is defined as the number of true positives divided by the total number of positive predictions. Precision is a good metric to use when we care more about avoiding false positives than false negatives.

Recall: This metric measures how many of the positive examples in the dataset are correctly predicted by the model. It is defined as the number of true positives divided by the total number of actual positive examples. Recall is a good metric to use when we care more about avoiding false negatives than false positives.

F1 score: This metric is a weighted average of precision and recall, where the weight is determined by the beta parameter. The most common value for beta is 1, which gives equal weight to precision and recall. The F1 score is a good metric to use when we want to balance precision and recall, and when the classes are imbalanced.

# 1. Import Label Dataset

In [ ]:
tag_climate_df = pd.read_parquet("Climate_Labels_Dataset.parquet")
tag_climate_df.head(5)

In [ ]:
#Only keep the required columns
tag_climate_df = tag_climate_df[["Text", "Final_Climate_Change_Level_Label"]]

In [ ]:
#Clean the tabel
tag_climate_df['Final_Climate_Change_Level_Label'] = tag_climate_df['Final_Climate_Change_Level_Label'].str.strip()
tag_climate_df[tag_climate_df["Final_Climate_Change_Level_Label"] == "NA"] = "Na"
tag_climate_df[tag_climate_df["Final_Climate_Change_Level_Label"] == "0"] = "Na"
tag_climate_df["Target"] = tag_climate_df["Final_Climate_Change_Level_Label"].apply(lambda x: "Yes" if x in ["High", "Medium"] else "No")

In [ ]:
overview_labels_hms = tag_climate_df.groupby("Final_Climate_Change_Level_Label")["Text"].count().reset_index()

In [ ]:
overview_labels_hms

In [ ]:
overview_labels = tag_climate_df.groupby("Target")["Text"].count().reset_index()

In [ ]:
overview_labels

In [ ]:
#overview_labels_hms.to_csv("C:/Users/Boedt/OneDrive/Bureaublad/R Thesis/overview_tag_labels_hms", index = False)
#overview_labels.to_csv("C:/Users/Boedt/OneDrive/Bureaublad/R Thesis/overview_tag_labels", index = False)

# 2. Taggers

## 2.1. Lexicon

### Global Change Lexicon

Uitleg

In [170]:
#Load the lexicon
Global_Change_Lexicon = pd.read_csv("Global_Change_Lexicon")
Global_Change_Lexicon = Global_Change_Lexicon.drop_duplicates().reset_index(drop = True)
Global_Change_Lexicon["Lexicon"] = Global_Change_Lexicon["Lexicon"].str.lower()

Global_Change_Lexicon

,Lexicon
0,100-year flood
1,emissions scenario
2,adaptation
3,adaptation science
4,adaptive capacity
...,...
100,vector
101,vulnerability
102,vulnerability assessment
103,water security


### IPCC Lexicon

Uitleg

In [215]:
#Load the lexicon
IPCC_Lexicon = pd.read_csv("IPCC_Lexicon")
IPCC_Lexicon = IPCC_Lexicon.drop_duplicates().reset_index(drop = True)
IPCC_Lexicon["Lexicon"] = IPCC_Lexicon["Lexicon"].str.lower()

IPCC_Lexicon

,Lexicon,Afkorting
0,acceptability of policy or system change,No
1,adaptability,No
2,adaptation,No
3,adaptation behaviour,No
4,adaptation limits,No
...,...,...
400,sd,Yes
401,sdgs,Yes
402,tcre,Yes
403,tod,Yes


### Wikipedia Lexicon

Uitleg

In [201]:
#Load the lexicon
Wikipedia_Lexicon = pd.read_csv("Wikipedia_Lexicon")
Wikipedia_Lexicon = Wikipedia_Lexicon.drop_duplicates().reset_index(drop = True)
Wikipedia_Lexicon["Lexicon"] = Wikipedia_Lexicon["Lexicon"].str.lower()

Wikipedia_Lexicon

,Lexicon
0,"100,000-year problem"
1,adaptation
2,additionality
3,albedo
4,anoxic event
...,...
159,volcanism
160,water vapor
161,weather
162,world climate report


### EPA Lexicon

Uitleg

In [224]:
#Load the lexicon
EPA_Lexicon = pd.read_csv("EPA_Lexicon")
EPA_Lexicon = EPA_Lexicon.drop_duplicates().reset_index(drop = True)
EPA_Lexicon["Lexicon"] = EPA_Lexicon["Lexicon"].str.lower()

EPA_Lexicon

list(EPA_Lexicon["Lexicon"])

['abrupt climate change',
 'adaptation',
 'adaptive capacity',
 'aerosols',
 'afforestation',
 'albedo',
 'alternative energy',
 'annex i countries/parties',
 'anthropogenic',
 'atmosphere',
 'atmospheric lifetime',
 'biofuels',
 'biogeochemical cycle',
 'biomass',
 'biosphere',
 'black carbon aerosol',
 'borehole',
 'carbon cycle',
 'carbon dioxide',
 'carbon dioxide equivalent',
 'carbon dioxide fertilization',
 'carbon footprint',
 'carbon sequestration',
 'carbon capture and sequestration',
 'chlorofluorocarbons',
 'climate',
 'climate change',
 'climate feedback',
 'climate lag',
 'climate model',
 'climate sensitivity',
 'climate system',
 'coal mine methane',
 'coalbed methane',
 'co-benefit',
 'concentration',
 'conference of the parties',
 'coral bleaching',
 'cryosphere',
 'deforestation',
 'desertification',
 'dryland farming',
 'eccentricity',
 'ecosystem',
 'el niño - southern oscillation',
 'emissions',
 'emissions factor',
 'energy efficiency',
 'energy star',
 'enhanced

### BBC Lexicon

Uitleg

In [225]:
#Load the lexicon
BBC_Lexicon = pd.read_csv("BBC_Lexicon")
BBC_Lexicon = BBC_Lexicon.drop_duplicates().reset_index(drop = True)
BBC_Lexicon["Lexicon"] = BBC_Lexicon["Lexicon"].str.lower()

BBC_Lexicon

,Lexicon,Afkorting
0,adaptation,No
1,adaptation fund,No
2,annex i countries,No
3,annex ii,No
4,anthropogenic climate change,No
...,...,...
74,gwp,Yes
75,ghgs,Yes
76,ji,Yes
77,350/450,No


### UNDP Lexicon

Uitleg

In [226]:
#Load the lexicon
UNDP_Lexicon = pd.read_csv("UNDP_Lexicon")
UNDP_Lexicon = UNDP_Lexicon.drop_duplicates().reset_index(drop = True)
UNDP_Lexicon["Lexicon"] = UNDP_Lexicon["Lexicon"].str.lower()

UNDP_Lexicon

,Lexicon,Afkorting
0,weather,No
1,climate,No
2,greenhouse gases,No
3,greenhouse gas emmisions,No
4,global warming,No
5,climate change,No
6,climate crisis,No
7,feedback loop,No
8,tipping point,No
9,climate overshoot,No


### Compare the lexicons to each other

In [231]:
#Create an empty dataframe and write a function to fill with the values

common_words_df = pd.DataFrame({"Lexicon" : ["Global Change", "IPCC", "Wikipedia", "EPA", "BBC", "UNDP"], 
                               "Global Change": [0, 0, 0, 0, 0, 0], "IPCC" : [0, 0, 0, 0, 0, 0], 
                               "Wikipedia" : [0, 0, 0, 0, 0, 0], "EPA" : [0, 0, 0, 0, 0, 0], 
                               "BBC" : [0, 0, 0, 0, 0, 0], "UNDP" : [0, 0, 0, 0, 0, 0]})

dfs = [Global_Change_Lexicon, IPCC_Lexicon, Wikipedia_Lexicon, EPA_Lexicon, BBC_Lexicon, UNDP_Lexicon]

for r in range(0, len(dfs)):
    for c in range(0, len(dfs)):
        # Get the common values between the two columns
        common_words = set(dfs[r]['Lexicon']).intersection(set(dfs[c]['Lexicon']))
        common_words_df.loc[r, common_words_df.columns[c +1]] = len(common_words)

common_words_df

,Lexicon,Global Change,IPCC,Wikipedia,EPA,BBC,UNDP
0,Global Change,105,38,12,20,11,7
1,IPCC,38,405,28,46,22,19
2,Wikipedia,12,28,164,33,15,10
3,EPA,20,46,33,176,16,14
4,BBC,11,22,15,16,79,11
5,UNDP,7,19,10,14,11,47


## 2.2. Hugging Face

In [ ]:
def summary(df_with_text, name, model_name, max_lenght_input=-1):
    data_in_list = df_with_text[name].tolist()
    tokenizer_sum = AutoTokenizer.from_pretrained(model_name)
    model_sum = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    summarizer = pipeline('summarization', model=model_sum, tokenizer = tokenizer_sum) 

    if max_lenght_input>=0:
        df_with_text['summary'] = summarizer(data_in_list, max_length=max_lenght_input)

    else:
        df_with_text['summary'] = summarizer(data_in_list)

def classification(df_with_text, name, model_name, max_lenght_input=-1):
    data_in_list = df_with_text[name].tolist()
    tokenizer_clas = AutoTokenizer.from_pretrained(model_name)
    model_clas = AutoModelForSequenceClassification.from_pretrained(model_name)
    classification = pipeline('text-classification', model=model_clas, tokenizer = tokenizer_clas) 

    if max_lenght_input>=0:
        df_with_text['classification'] = classification(data_in_list, max_length=max_lenght_input, truncation=True)

    else:
        df_with_text['classification'] = classification(data_in_list)

In [ ]:
#classification(Lexicon_df, 'Text',"climatebert/environmental-claims",512)

# 3. Testen Taggers

## 3.1. Lexicon

In [232]:
#create a separate df with the specific cleaning for the lexicons
Lexicon_df = tag_climate_df.copy()
Lexicon_df["Text"] = Lexicon_df["Text"].apply(preprocess_text)

### 3.1.1. One Lexicon

In [233]:
#Global Change Lexicon
Global_Change_df_1 = get_metrics_df_1(Lexicon_df, Global_Change_Lexicon, 0, 20, "Global Change")
Global_Change_df_2 = get_metrics_df_2(Lexicon_df, Global_Change_Lexicon, 0, 2, 0.1, "Global Change")
Global_Change_df_3 = get_metrics_df_3(Lexicon_df, Global_Change_Lexicon, 0, 20, "Global Change")
Global_Change_df_4 = get_metrics_df_4(Lexicon_df, Global_Change_Lexicon, 0, 2, 0.1, "Global Change")

In [234]:
Global_Change_df = pd.concat([Global_Change_df_1, Global_Change_df_2, Global_Change_df_3, Global_Change_df_4])

In [235]:
#IPCC Lexicon
IPCC_df_1 = get_metrics_df_1(Lexicon_df, IPCC_Lexicon, 0, 20, "IPCC")
IPCC_df_2 = get_metrics_df_2(Lexicon_df, IPCC_Lexicon, 0, 2, 0.1, "IPCC")
IPCC_df_3 = get_metrics_df_3(Lexicon_df, IPCC_Lexicon, 0, 20, "IPCC")
IPCC_df_4 = get_metrics_df_4(Lexicon_df, IPCC_Lexicon, 0, 2, 0.1, "IPCC")

In [236]:
IPCC_df = pd.concat([IPCC_df_1, IPCC_df_2, IPCC_df_3, IPCC_df_4])

In [237]:
#Wikipedia Lexicon
Wikipedia_df_1 = get_metrics_df_1(Lexicon_df, Wikipedia_Lexicon, 0, 20, "Wikipedia")
Wikipedia_df_2 = get_metrics_df_2(Lexicon_df, Wikipedia_Lexicon, 0, 2, 0.1, "Wikipedia")
Wikipedia_df_3 = get_metrics_df_3(Lexicon_df, Wikipedia_Lexicon, 0, 20, "Wikipedia")
Wikipedia_df_4 = get_metrics_df_4(Lexicon_df, Wikipedia_Lexicon, 0, 2, 0.1, "Wikipedia")

In [238]:
Wikipedia_df = pd.concat([Wikipedia_df_1, Wikipedia_df_2, Wikipedia_df_3, Wikipedia_df_4])

In [ ]:
#EPA Lexicon
EPA_df_1 = get_metrics_df_1(Lexicon_df, EPA_Lexicon, 0, 20, "EPA")
EPA_df_2 = get_metrics_df_2(Lexicon_df, EPA_Lexicon, 0, 2, 0.1, "EPA")
EPA_df_3 = get_metrics_df_3(Lexicon_df, EPA_Lexicon, 0, 20, "EPA")
EPA_df_4 = get_metrics_df_4(Lexicon_df, EPA_Lexicon, 0, 2, 0.1, "EPA")

In [ ]:
EPA_df = pd.concat([EPA_df_1, EPA_df_2, EPA_df_3, EPA_df_4])

In [ ]:
#BBC Lexicon
BBC_df_1 = get_metrics_df_1(Lexicon_df, BBC_Lexicon, 0, 20, "BBC")
BBC_df_2 = get_metrics_df_2(Lexicon_df, BBC_Lexicon, 0, 2, 0.1, "BBC")
BBC_df_3 = get_metrics_df_3(Lexicon_df, BBC_Lexicon, 0, 20, "BBC")
BBC_df_4 = get_metrics_df_4(Lexicon_df, BBC_Lexicon, 0, 2, 0.1, "BBC")

In [ ]:
BBC_df = pd.concat([BBC_df_1, BBC_df_2, BBC_df_3, BBC_df_4])

In [ ]:
threshold_metrics_2(Lexicon_df, UNDP_Lexicon, 0.6, 0.8, 0.1)

In [ ]:
count_lexicon_words_1(Lexicon_df, UNDP_Lexicon)

In [ ]:
#UNDP Lexicon
UNDP_df_1 = get_metrics_df_1(Lexicon_df, UNDP_Lexicon, 0, 20, "UNDP")
UNDP_df_2 = get_metrics_df_2(Lexicon_df, UNDP_Lexicon, 0, 2, 0.1, "UNDP")
UNDP_df_3 = get_metrics_df_3(Lexicon_df, UNDP_Lexicon, 0, 20, "UNDP")
UNDP_df_4 = get_metrics_df_4(Lexicon_df, UNDP_Lexicon, 0, 2, 0.1, "UNDP")

In [ ]:
UNDP_df = pd.concat([UNDP_df_1, UNDP_df_2, UNDP_df_3, UNDP_df_4])

In [ ]:
#Get all lexicons together
Lexicon_df_1 = pd.concat([Global_Change_df, IPCC_df, Wikipedia_df, EPA_df, BBC_df, UNDP_df])

In [ ]:
Lexicon_df_1.sort_values("Accuracy", ascending = False).reset_index(drop = True).head(20)

### 3.1.2. Two Lexicons Combined

In [ ]:
#UNDP and EPA
EPA_UNDP_Lexicon = pd.concat([EPA_Lexicon, UNDP_Lexicon]).drop_duplicates().reset_index(drop = True)
EPA_UNDP_df_1 = get_metrics_df_1(Lexicon_df, EPA_UNDP_Lexicon, 0, 20, "EPA_UDNP")
EPA_UNDP_df_2 = get_metrics_df_2(Lexicon_df, EPA_UNDP_Lexicon, 0, 2, 0.1, "EPA_UDNP")
EPA_UNDP_df_3 = get_metrics_df_3(Lexicon_df, EPA_UNDP_Lexicon, 0, 20, "EPA_UDNP")
EPA_UNDP_df_4 = get_metrics_df_4(Lexicon_df, EPA_UNDP_Lexicon, 0, 2, 0.1, "EPA_UDNP")

In [ ]:
EPA_UNDP_df = pd.concat([EPA_UNDP_df_1, EPA_UNDP_df_2, EPA_UNDP_df_3, EPA_UNDP_df_4])

In [ ]:
#UNDP and BBC
BBC_UNDP_Lexicon = pd.concat([BBC_Lexicon, UNDP_Lexicon]).drop_duplicates().reset_index(drop = True)
BBC_UNDP_df_1 = get_metrics_df_1(Lexicon_df, BBC_UNDP_Lexicon, 0, 20, "BBC_UNDP")
BBC_UNDP_df_2 = get_metrics_df_2(Lexicon_df, BBC_UNDP_Lexicon, 0, 2, 0.1, "BBC_UNDP")
BBC_UNDP_df_3 = get_metrics_df_3(Lexicon_df, BBC_UNDP_Lexicon, 0, 20, "BBC_UNDP")
BBC_UNDP_df_4 = get_metrics_df_4(Lexicon_df, BBC_UNDP_Lexicon, 0, 2, 0.1, "BBC_UNDP")

In [ ]:
BBC_UNDP_df = pd.concat([BBC_UNDP_df_1, BBC_UNDP_df_2, BBC_UNDP_df_3, BBC_UNDP_df_4])

In [ ]:
#UNDP and Global Change 
UNDP_Global_Change_Lexicon = pd.concat([Global_Change_Lexicon, UNDP_Lexicon]).drop_duplicates().reset_index(drop = True)
UNDP_Global_Change_df_1 = get_metrics_df_1(Lexicon_df, UNDP_Global_Change_Lexicon, 0, 20, "UNDP_Global_Change")
UNDP_Global_Change_df_2 = get_metrics_df_2(Lexicon_df, UNDP_Global_Change_Lexicon, 0, 2, 0.1, "UNDP_Global_Change")
UNDP_Global_Change_df_3 = get_metrics_df_3(Lexicon_df, UNDP_Global_Change_Lexicon, 0, 20, "UNDP_Global_Change")
UNDP_Global_Change_df_4 = get_metrics_df_4(Lexicon_df, UNDP_Global_Change_Lexicon, 0, 2, 0.1, "UNDP_Global_Change")

In [ ]:
Global_Change_UNDP_df = pd.concat([UNDP_Global_Change_df_1, UNDP_Global_Change_df_2, UNDP_Global_Change_df_3, UNDP_Global_Change_df_4])

In [ ]:
Global_Change_UNDP_df.sort_values("Accuracy", ascending = False)

In [ ]:
#UNDP and IPCC
UNDP_IPCC_Lexicon = pd.concat([IPCC_Lexicon, UNDP_Lexicon]).drop_duplicates().reset_index(drop = True)
UNDP_IPCC_df_1 = get_metrics_df_1(Lexicon_df, UNDP_IPCC_Lexicon, 0, 20, "UNDP_IPCC")
UNDP_IPCC_df_2 = get_metrics_df_2(Lexicon_df, UNDP_IPCC_Lexicon, 0, 2, 0.1, "UNDP_IPCC")
UNDP_IPCC_df_3 = get_metrics_df_3(Lexicon_df, UNDP_IPCC_Lexicon, 0, 20, "UNDP_IPCC")
UNDP_IPCC_df_4 = get_metrics_df_4(Lexicon_df, UNDP_IPCC_Lexicon, 0, 2, 0.1, "UNDP_IPCC")

In [ ]:
IPCC_UNDP_df = pd.concat([UNDP_IPCC_df_1, UNDP_IPCC_df_2, UNDP_IPCC_df_3, UNDP_IPCC_df_4])

In [ ]:
#UNDP and Wikipedia
UNDP_Wikipedia_Lexicon = pd.concat([Wikipedia_Lexicon, UNDP_Lexicon]).drop_duplicates().reset_index(drop = True)
UNDP_Wikipedia_df_1 = get_metrics_df_1(Lexicon_df, UNDP_Wikipedia_Lexicon, 0, 20, "UNDP_Wikipedia")
UNDP_Wikipedia_df_2 = get_metrics_df_2(Lexicon_df, UNDP_Wikipedia_Lexicon, 0, 2, 0.1, "UNDP_Wikipedia")
UNDP_Wikipedia_df_3 = get_metrics_df_3(Lexicon_df, UNDP_Wikipedia_Lexicon, 0, 20, "UNDP_Wikipedia")
UNDP_Wikipedia_df_4 = get_metrics_df_4(Lexicon_df, UNDP_Wikipedia_Lexicon, 0, 2, 0.1, "UNDP_Wikipedia")

In [ ]:
Wikipedia_UNDP_df = pd.concat([UNDP_Wikipedia_df_1, UNDP_Wikipedia_df_2, UNDP_Wikipedia_df_3, UNDP_Wikipedia_df_4])

In [ ]:
Wikipedia_UNDP_df.sort_values("Accuracy", ascending = False)

In [ ]:
Lexicon_df_2 = pd.concat([EPA_UNDP_df, BBC_UNDP_df, Global_Change_UNDP_df, IPCC_UNDP_df, Wikipedia_UNDP_df]).reset_index(drop = True)

In [ ]:
Lexicon_df_2.sort_values("Accuracy", ascending = False)

In [ ]:
#EPA and BBC
EPA_BBC_Lexicon = pd.concat([EPA_Lexicon, BBC_Lexicon]).drop_duplicates().reset_index(drop = True)
EPA_BBC_df_1 = get_metrics_df_1(Lexicon_df, EPA_BBC_Lexicon, 0, 20, "BBC_EPA")
EPA_BBC_df_2 = get_metrics_df_2(Lexicon_df, EPA_BBC_Lexicon, 0, 2, 0.1, "BBC_EPA")
EPA_BBC_df_3 = get_metrics_df_3(Lexicon_df, EPA_BBC_Lexicon, 0, 20, "BBC_EPA")
EPA_BBC_df_4 = get_metrics_df_4(Lexicon_df, EPA_BBC_Lexicon, 0, 2, 0.1, "BBC_EPA")

In [ ]:
EPA_BBC_df = pd.concat([EPA_BBC_df_1, EPA_BBC_df_2, EPA_BBC_df_3, EPA_BBC_df_4])

In [ ]:
EPA_BBC_df.sort_values("Accuracy", ascending = False)

In [ ]:
#EPA and Global Change
EPA_Global_Change_Lexicon = pd.concat([EPA_Lexicon, Global_Change_Lexicon]).drop_duplicates().reset_index(drop = True)
EPA_Global_Change_df_1 = get_metrics_df_1(Lexicon_df, EPA_Global_Change_Lexicon, 0, 20, "EPA_GLobal_Change")
EPA_Global_Change_df_2 = get_metrics_df_2(Lexicon_df, EPA_Global_Change_Lexicon, 0, 2, 0.1, "BBC_Global_Change")
EPA_Global_Change_df_3 = get_metrics_df_3(Lexicon_df, EPA_Global_Change_Lexicon, 0, 20, "BBC_Global_Change")
EPA_Global_Change_df_4 = get_metrics_df_4(Lexicon_df, EPA_Global_Change_Lexicon, 0, 2, 0.1, "BBC_Global_Change")

In [ ]:
EPA_Global_Change_df = pd.concat([EPA_Global_Change_df_1, EPA_Global_Change_df_2, EPA_Global_Change_df_3, EPA_Global_Change_df_4])

In [ ]:
EPA_Global_Change_df.sort_values("Accuracy", ascending = False)

In [ ]:
#EPA and IPCC
EPA_IPCC_Lexicon = pd.concat([EPA_Lexicon, IPCC_Lexicon]).drop_duplicates().reset_index(drop = True)
EPA_IPCC_df_1 = get_metrics_df_1(Lexicon_df, EPA_IPCC_Lexicon, 0, 20, "EPA_IPCC")
EPA_IPCC_df_2 = get_metrics_df_2(Lexicon_df, EPA_IPCC_Lexicon, 0, 2, 0.1, "EPA_IPCC")
EPA_IPCC_df_3 = get_metrics_df_3(Lexicon_df, EPA_IPCC_Lexicon, 0, 20, "EPA_IPCC")
EPA_IPCC_df_4 = get_metrics_df_4(Lexicon_df, EPA_IPCC_Lexicon, 0, 2, 0.1, "EPA_IPCC")

In [ ]:
EPA_IPCC_df = pd.concat([EPA_IPCC_df_1, EPA_IPCC_df_2, EPA_IPCC_df_3, EPA_IPCC_df_4])

In [ ]:
EPA_IPCC_df.sort_values("Accuracy", ascending = False)

In [ ]:
#EPA and Wikipedia
EPA_Wikipedia_Lexicon = pd.concat([EPA_Lexicon, Wikipedia_Lexicon]).drop_duplicates().reset_index(drop = True)
EPA_Wikipedia_df_1 = get_metrics_df_1(Lexicon_df, EPA_Wikipedia_Lexicon, 0, 20, "EPA_Wikipedia")
EPA_Wikipedia_df_2 = get_metrics_df_2(Lexicon_df, EPA_Wikipedia_Lexicon, 0, 2, 0.1, "EPA_Wikipedia")
EPA_Wikipedia_df_3 = get_metrics_df_3(Lexicon_df, EPA_Wikipedia_Lexicon, 0, 20, "EPA_Wikipedia")
EPA_Wikipedia_df_4 = get_metrics_df_4(Lexicon_df, EPA_Wikipedia_Lexicon, 0, 2, 0.1, "EPA_Wikipedia")

In [ ]:
EPA_Wikipedia_df = pd.concat([EPA_Wikipedia_df_1, EPA_Wikipedia_df_2, EPA_Wikipedia_df_3, EPA_Wikipedia_df_4])

In [ ]:
EPA_Wikipedia_df.sort_values("Accuracy", ascending = False)

In [ ]:
Lexicon_df_2 = pd.concat([Lexicon_df_2, EPA_BBC_df, EPA_Global_Change_df, EPA_IPCC_df, EPA_Wikipedia_df]).reset_index(drop = True)

In [ ]:
Lexicon_df_2.sort_values("Accuracy", ascending = False)

In [ ]:
#BBC and Global Change
#EPA and Wikipedia
BBC_Global_Change_Lexicon = pd.concat([BBC_Lexicon, Global_Change_Lexicon]).drop_duplicates().reset_index(drop = True)
BBC_Global_Change_df_1 = get_metrics_df_1(Lexicon_df, BBC_Global_Change_Lexicon, 0, 20, "BBC_Global_Change")
BBC_Global_Change_df_2 = get_metrics_df_2(Lexicon_df, BBC_Global_Change_Lexicon, 0, 2, 0.1, "BBC_Global_Change")
BBC_Global_Change_df_3 = get_metrics_df_3(Lexicon_df, BBC_Global_Change_Lexicon, 0, 20, "BBC_Global_Change")
BBC_Global_Change_df_4 = get_metrics_df_4(Lexicon_df, BBC_Global_Change_Lexicon, 0, 2, 0.1, "BBC_Global_Change")

In [ ]:
BBC_Global_Change_df = pd.concat([BBC_Global_Change_df_1, BBC_Global_Change_df_2, BBC_Global_Change_df_3, BBC_Global_Change_df_4])

In [ ]:
#BBC and IPCC
BBC_IPCC_Lexicon = pd.concat([BBC_Lexicon, IPCC_Lexicon]).drop_duplicates().reset_index(drop = True)
BBC_IPCC_df_1 = get_metrics_df_1(Lexicon_df, BBC_IPCC_Lexicon, 0, 20, "BBC_IPCC")
BBC_IPCC_df_2 = get_metrics_df_2(Lexicon_df, BBC_IPCC_Lexicon, 0, 2, 0.1, "BBC_IPCC")
BBC_IPCC_df_3 = get_metrics_df_3(Lexicon_df, BBC_IPCC_Lexicon, 0, 20, "BBC_IPCC")
BBC_IPCC_df_4 = get_metrics_df_4(Lexicon_df, BBC_IPCC_Lexicon, 0, 2, 0.1, "BBC_IPCC")

In [ ]:
BBC_IPCC_df = pd.concat([BBC_IPCC_df_1, BBC_IPCC_df_2, BBC_IPCC_df_3, BBC_IPCC_df_4])

In [ ]:
#BBC and Wikipedia
BBC_Wikipedia_Lexicon = pd.concat([BBC_Lexicon, Wikipedia_Lexicon]).drop_duplicates().reset_index(drop = True)
BBC_Wikipedia_df_1 = get_metrics_df_1(Lexicon_df, BBC_Wikipedia_Lexicon, 0, 20, "BBC_Wikipedia")
BBC_Wikipedia_df_2 = get_metrics_df_2(Lexicon_df, BBC_Wikipedia_Lexicon, 0, 2, 0.1, "BBC_Wikipedia")
BBC_Wikipedia_df_3 = get_metrics_df_3(Lexicon_df, BBC_Wikipedia_Lexicon, 0, 20, "BBC_Wikipedia")
BBC_Wikipedia_df_4 = get_metrics_df_4(Lexicon_df, BBC_Wikipedia_Lexicon, 0, 2, 0.1, "BBC_Wikipedia")

In [ ]:
BBC_Wikpedia_df = pd.concat([BBC_Wikipedia_df_1, BBC_Wikipedia_df_2, BBC_Wikipedia_df_3, BBC_Wikipedia_df_4])

In [ ]:
Lexicon_df_2 = pd.concat([Lexicon_df_2, BBC_Global_Change_df, BBC_IPCC_df, BBC_Wikpedia_df]).reset_index(drop = True)

In [ ]:
#Wikipedia and Global Change
Wikipedia_Global_Change_Lexicon = pd.concat([Global_Change_Lexicon, Wikipedia_Lexicon]).drop_duplicates().reset_index(drop = True)
Wikipedia_Global_Change_df_1 = get_metrics_df_1(Lexicon_df, Wikipedia_Global_Change_Lexicon, 0, 20, "Wikipedia_Global_Change")
Wikipedia_Global_Change_df_2 = get_metrics_df_2(Lexicon_df, Wikipedia_Global_Change_Lexicon, 0, 2, 0.1, "Wikipedia_Global_Change")
Wikipedia_Global_Change_df_3 = get_metrics_df_3(Lexicon_df, Wikipedia_Global_Change_Lexicon, 0, 20, "Wikipedia_Global_Change")
Wikipedia_Global_Change_df_4 = get_metrics_df_4(Lexicon_df, Wikipedia_Global_Change_Lexicon, 0, 2, 0.1, "Wikipedia_Global_Change")

In [ ]:
Wikpedia_Global_Change_df = pd.concat([Wikipedia_Global_Change_df_1, Wikipedia_Global_Change_df_2, Wikipedia_Global_Change_df_3, Wikipedia_Global_Change_df_4])

In [ ]:
#Wikipedia and IPCC
Wikipedia_IPCC_Lexicon = pd.concat([IPCC_Lexicon, Wikipedia_Lexicon]).drop_duplicates().reset_index(drop = True)
Wikipedia_IPCC_df_1 = get_metrics_df_1(Lexicon_df, Wikipedia_IPCC_Lexicon, 0, 20, "Wikipedia_IPCC")
Wikipedia_IPCC_df_2 = get_metrics_df_2(Lexicon_df, Wikipedia_IPCC_Lexicon, 0, 2, 0.1, "Wikipedia_IPCC")
Wikipedia_IPCC_df_3 = get_metrics_df_3(Lexicon_df, Wikipedia_IPCC_Lexicon, 0, 20, "Wikipedia_IPCC")
Wikipedia_IPCC_df_4 = get_metrics_df_4(Lexicon_df, Wikipedia_IPCC_Lexicon, 0, 2, 0.1, "Wikipedia_IPCC")

In [ ]:
Wikpedia_IPCC_df = pd.concat([Wikipedia_IPCC_df_1, Wikipedia_IPCC_df_2, Wikipedia_IPCC_df_3, Wikipedia_IPCC_df_4])

In [ ]:
#IPCC and Global Change
Global_Change_IPCC_Lexicon = pd.concat([IPCC_Lexicon, Global_Change_Lexicon]).drop_duplicates().reset_index(drop = True)
Global_Change_IPCC_df_1 = get_metrics_df_1(Lexicon_df, Global_Change_IPCC_Lexicon, 0, 20, "Global_Change_IPCC")
Global_Change_IPCC_df_2 = get_metrics_df_2(Lexicon_df, Global_Change_IPCC_Lexicon, 0, 2, 0.1, "Global_Change_IPCC")
Global_Change_IPCC_df_3 = get_metrics_df_3(Lexicon_df, Global_Change_IPCC_Lexicon, 0, 20, "Global_Change_IPCC")
Global_Change_IPCC_df_4 = get_metrics_df_4(Lexicon_df, Global_Change_IPCC_Lexicon, 0, 2, 0.1, "Global_Change_IPCC")

In [ ]:
Global_Change_IPCC_df = pd.concat([Global_Change_IPCC_df_1, Global_Change_IPCC_df_2, Global_Change_IPCC_df_3, Global_Change_IPCC_df_4])

In [ ]:
Lexicon_df_2 = pd.concat([Lexicon_df_2, Wikpedia_Global_Change_df, Wikpedia_IPCC_df, Global_Change_IPCC_df]).reset_index(drop = True)

In [ ]:
Lexicon_df_2.sort_values("Precision", ascending = False)

In [ ]:
Lexicon_df = pd.concat([Lexicon_df_1, Lexicon_df_2]).reset_index(drop = True)

In [ ]:
Lexicon_df.sort_values(by="Accuracy", ascending = False)